### Notebook to calculate NDSI and snow-covered area (SCA) in PlanetScope 4-band imagery

Rainey Aberle

Spring 2022

In [ ]:
# Import packages
import os
import glob
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
import earthpy.spatial as es
import matplotlib
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from shapely.geometry import shape

In [ ]:
# -----Define paths in directory
# base directory
base_path = '/Users/raineyaberle/Research/PhD/study-sites/Wolverine/'
# image directory
im_path = base_path+'imagery/Planet/2021-04-20_2021-08-25/adjusted-radiometry/'
# figures output folder
figures_out_path = base_path+'figures/SCA/'

# -----Determine whether to save output figures
save_figures = False # = True to save output figures

In [ ]:
# -----Define Area of Interest (AOI) for image cropping
# Read in shapefile with AOI polygon
fn = base_path+'../../GIS_data/wolverine_RGI.shp'
#fn = basepath+'GIS_data/USGS/wolverine_boundary/Wolverine_Glacier_Boundaries.shp'
AOI = gpd.read_file(fn)
# Reproject to imagery CRS if necessary
AOI = AOI.to_crs(32606)

In [ ]:
# -----Load image names from file
os.chdir(im_path) # change directory
im_names = glob.glob('*.tif') # load all .tif file names
im_names.sort() # sort file names by date

In [ ]:
# -----Crop images to AOI

# make folder for cropped images if it does not exist
cropped_im_path = im_path+'../cropped/'
if os.path.isdir(cropped_im_path)==0:
    os.mkdir(cropped_im_path)
    print(cropped_im_path+' directory made')

# loop through images
for im_name in im_names:
    
    # open image
    im = rio.open(im_name)    
    
    # check if file exists in file already
    cropped_im_fn = cropped_im_path + im_name[0:8] + '_crop.tif'
    if os.path.exists(cropped_im_fn)==True:
        print('cropped image already exists in directory')
    else:
        # mask image pixels outside the AOI
        AOI_bb = [AOI.bounds]
        out_image, out_transform = mask(im, list(AOI.geometry), crop=True)
        out_meta = im.meta.copy()
        out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
        with rio.open(cropped_im_fn, "w", **out_meta) as dest:
            dest.write(out_image)
        print(cropped_im_fn+' saved')

In [ ]:
# -----Calculate NDSI and Snow-Covered Area

# grab cropped image names
os.chdir(cropped_im_path) # change directory
im_names_crop = glob.glob('*_crop.tif')
im_names_crop.sort() # sort file names by date

# loop through cropped images
dates=[]
snow_areas=[]
for im_name in im_names_crop:
    
    # open image
    im = rio.open(im_name)

    # extract date from image name
    date = im_name[0:4] + '-' + im_name[4:6] + '-' + im_name[6:8]
    dates = dates + [np.datetime64(date)]

    # calculate area of snow 
    # define bands, convert to TOA reflectance
    b = im.read(1).astype(float)
    g = im.read(2).astype(float)
    r = im.read(3).astype(float)
    nir = im.read(4).astype(float)
    # compute NDSI
    ndsi = es.normalized_diff(r, nir) 
    
    # define coordinates grid
    x = np.linspace(im.bounds.left, im.bounds.right, num=np.shape(b)[1])
    y = np.linspace(im.bounds.top, im.bounds.bottom, num=np.shape(b)[0])
    
    # determine thresholds 
    ndsi_thresh = [0, 0.2]
    r_thresh = 0.7

    # threshold NDSI to determine snow cover
    snow = np.where((ndsi>np.min(ndsi_thresh)) & (ndsi<np.max(ndsi_thresh)) & (r > r_thresh),1,0)
    
    # calculate total area of snow
    pA = im.res[0]*im.res[1] # pixel area [m^2]
    snow_count = np.count_nonzero(snow) # number of snow pixels
    snow_area = pA * snow_count /10e3 # area of snow [km^2]
    snow_areas = snow_areas + [snow_area]
    
    # plot 
    fig = plt.figure(figsize=(12,10))
    plt.rcParams.update({'font.size': 14, 'font.sans-serif': 'Arial'})
    gs = GridSpec(3, 2, figure=fig)
    # RGB image
    fig.suptitle(date)
    ax1 = fig.add_subplot(gs[0:2, 0])
    ax1.imshow(np.dstack([r, g, b]), 
               extent=(np.min(x)/1000, np.max(x)/1000, np.min(y)/1000, np.max(y)/1000))
    ax1.set_xlabel('Easting [km]')
    ax1.set_ylabel('Northing [km]')
    # snow
    ax2 = fig.add_subplot(gs[0:2, 1])
    ax2.imshow(snow, cmap='Blues', clim=(0,1), 
               extent=(np.min(x)/1000, np.max(x)/1000, np.min(y)/1000, np.max(y)/1000))
    ax2.set_xlabel('Easting [km]')
    # SCA time series
    ax3 = fig.add_subplot(gs[2, :])
    ax3.grid()
    ax3.set_ylabel('Snow-Covered Area [km^2]')
    ax3.scatter(dates,snow_areas,color='blue')
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    # Rotate and right-align the x-tick labels
    for label in ax3.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    ax3.set_xlim(np.datetime64('2021-04-01'), np.datetime64('2021-09-01'))
    ax3.set_ylim(100,1700)
    fig.tight_layout()
    plt.show()
    
    # save figure
    if save_figures==True:
        fig.savefig(figures_out_path+date+'_SCA.png', dpi=200, facecolor='white', edgecolor='none')
        print('figure saved to file')


In [ ]:
# -----Plot results for one image with approximate snowline

## snow ##
# load cropped image
im_name_crop = glob.glob(cropped_im_path+im_name[0:-4]+'_crop.tif')
im_crop = rio.open(im_name_crop)
# define bands
g = im_crop.read(2) 
nir = im_crop.read(4)
# compute NDSI
ndsi = calculate_ndsi(nir,red) 
# threshold NDSI to determine snow cover
snow = np.where((ndsi>thresh_min) & (ndsi<thresh_max),1,np.nan)

# snowline
sl_fn = base_path+'../../GIS_data/digitized_snowline_picks/20210611_snowline.shp'
sl = gpd.read_file(sl_fn) # snowline
# Reproject to imagery CRS if necessary
sl = sl.to_crs(32606)

# transect 
ts_fn = base_path+'../../GIS_data/digitized_snowline_picks/transect.shp'
ts = gpd.read_file(ts_fn) # snowline
# Reproject to imagery CRS if necessary
ts = ts.to_crs(32606)
# extract points from geometry
ts_pts = [i for i in ts.geometry]
ts_x,ts_y = ts_pts[0].coords.xy
ts_coords = np.dstack((ts_x,ts_y)).tolist()
# sample raster values at coordinates
ts_rv = [x for x in im.sample(ts_coords[0])]
# calculate NDSI at points
ts_ndsi=[]
for s in ts_rv:
    ts_ndsi = ts_ndsi + [calculate_ndsi(s[3], s[1])]
    
# plot
fig = plt.figure(figsize=(15, 10))
plt.rcParams.update({'font.size': 14, 'font.sans-serif': 'Arial'})
# ax1
ax1 = plt.subplot(1,2,1)
ax1.set_xlabel('Easting [m]')
ax1.set_ylabel('Northing [m]')
show(snow, ax=ax1, transform=im_crop.transform, cmap='Blues', clim=(0.5,1.2), label='snow')
AOI.plot(ax=ax1,facecolor='none',edgecolor='black', label='Glacier Outline')
sl.plot(ax=ax1, edgecolor='maroon', label='Snowline')
ax1.scatter(ts_x, ts_y, color='blue', label='Transect')
ax1.legend()
# Rotate and right-align the x-tick labels
for label in ax1.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
# ax2
ax2 = plt.subplot(2,2,2)
ax2.plot(dates,snow_areas,'.-k',markersize=15)
I = np.where(dates==np.datetime64('2021-06-11'))[0][0]
ax2.plot(dates[I], snow_areas[I],'*m',markersize=18)
ax2.grid()
ax2.set_ylabel('Snow-Covered Area [km$^2$]')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
# Rotate and right-align the x-tick labels
for label in ax2.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
# ax3
ax3 = plt.subplot(2,2,4)
ax3.set_xlim(np.min(ts_y)-2e2, np.max(ts_y)+2e2)
rect=matplotlib.patches.Rectangle((ax3.get_xlim()[0],thresh_min),ax3.get_xlim()[1]-ax3.get_xlim()[0],
                                  thresh_max-thresh_min, fill=True, color="grey", alpha=0.2, label='threshold')
plt.gca().add_patch(rect)
ax3.scatter(ts_y, ts_ndsi, color='blue', label='Transect')
ax3.set_xlabel('Northing [m]')
ax3.set_ylabel('NDSI')
# Rotate and right-align the x-tick labels
for label in ax3.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
ax3.grid()
ax3.legend()
# add panel labels (a-c)
ax1.text(ax1.get_xlim()[0] + (ax1.get_xlim()[1] - ax1.get_xlim()[0])*0.05,
         ax1.get_ylim()[0] + (ax1.get_ylim()[1] - ax1.get_ylim()[0])*0.08,'a',
        fontsize=18, fontweight='bold')
ax2.text(ax2.get_xlim()[0] + (ax2.get_xlim()[1] - ax2.get_xlim()[0])*0.05,
         ax2.get_ylim()[0] + (ax2.get_ylim()[1] - ax2.get_ylim()[0])*0.08,'b',
        fontsize=18, fontweight='bold')
ax3.text(ax3.get_xlim()[0] + (ax3.get_xlim()[1] - ax3.get_xlim()[0])*0.05,
         ax3.get_ylim()[0] + (ax3.get_ylim()[1] - ax3.get_ylim()[0])*0.08,'c',
        fontsize=18, fontweight='bold')
plt.show()

# save figure
# fig.savefig(outpath+'SCA_threshold.png')
# print('figure saved.')